**2.1**

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}
---

New Constraint

$|(x_{1} + x_{2})-(x_{3} + x_{4}+x_{5})|$ <=18

Now the resultant formulation is not linear program as it contains absolute value function which are not linear.

1.2
New formualtion using approach 1

Let $u=|(x_{1} + x_{2})-(x_{3} + x_{4}+x_{5})|$

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}

$u<=18$  
$u - x_{1} - x_{2} + x_{3} + x_{4}+x_{5} >=0$   
$u + x_{1} + x_{2} - x_{3} - x_{4}-x_{5} >=0$  
$u>=0$ 

All the decision variables are positive integers.

1.2
New formualtion using approach 2

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}

New constraints:  
$(x_{1} + x_{2}) - (x_{3} + x_{4}+x_{5}) = a-b$   
$a+b <=18$ 

Here  a and b are non negative integers.


In [ ]:
!pip install -q pyomo
from pyomo.environ import * 
import numpy as np
import pandas as pd
!apt-get install -y -qq coinor-cbc

In [10]:
# 2.4 Solving problem from approach 1
data_csvfile = pd.read_csv('sample_csvfile.csv')

model_1 = ConcreteModel()

N = len(data_csvfile.index)
M = 5
col_indices = range(N)

#defining variables
model_1.x = Var(col_indices, domain=NonNegativeIntegers)
model_1.u = Var(domain=NonNegativeIntegers)

#defining objective
model_1.objective = Objective(expr=summation(data_csvfile['Reach Quality'], model_1.x),sense=maximize)


#adding constraints
model_1.constraints = ConstraintList()

model_1.constraints.add(expr= model_1.x[0] + model_1.x[1] + model_1.x[2]<= 75  )
model_1.constraints.add(expr= model_1.x[2] + model_1.x[3] +  model_1.x[4] <= 100  )
model_1.constraints.add(expr= model_1.x[0] + model_1.x[1] >= 10  )

model_1.constraints.add(summation(data_csvfile['Cost per advt'], model_1.x) <= 1500000)

model_1.constraints.add(summation(data_csvfile['Num Customers'], model_1.x) >= 900000)

#additional constraints
model_1.constraints.add(expr=model_1.u<=18)
model_1.constraints.add(expr=model_1.u-model_1.x[0]-model_1.x[1]+model_1.x[2]+model_1.x[3]+model_1.x[4]>=0)
model_1.constraints.add(expr=model_1.u+model_1.x[0]+model_1.x[1]-model_1.x[2]-model_1.x[3]-model_1.x[4]>=0)

model_1.pprint()


2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {1, 2, 3, 4, 5, 6, 7, 8}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

2 Var Declarations
    u : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers

1 Objective 

In [11]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(model_1)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


2.4  By Approach 1
  
Solver status is OK  
Solver termination condition is optimal  
We are getting optimal solution by approach 1


In [12]:
print('\nObjective = ', model_1.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num slots for medium ',data_csvfile['Medium'][i], model_1.x[i].value)

print("value of new variable u=",model_1.u.value)

print('\nConstraints')
model_1.constraints.display()


Objective =  3378.0

Decision Variables
Num slots for medium  Websites 34.0
Num slots for medium  TV 0.0
Num slots for medium  DailyNewspaper 41.0
Num slots for medium  WeeklyMagazine 11.0
Num slots for medium  MonthlyMagazine 0.0
value of new variable u= 18.0

Constraints
constraints : Size=8
    Key : Lower    : Body       : Upper
      1 :     None :       75.0 :      75.0
      2 :     None :       52.0 :     100.0
      3 :     10.0 :       34.0 :      None
      4 :     None :  1485000.0 : 1500000.0
      5 : 900000.0 : 12095000.0 :      None
      6 :     None :       18.0 :      18.0
      7 :      0.0 :       36.0 :      None
      8 :      0.0 :        0.0 :      None


2.5 By approach 1

Optimal objective value and value of decision variables and new introduced varible are shown above.

Constraint activity.  
Constraint 1,4,6 and 8 are active. Other constraints are inactive.

In [16]:
# 2.4 solving problem by approach 2

data_csvfile = pd.read_csv('sample_csvfile.csv')

model_2 = ConcreteModel()

N = len(data_csvfile.index)
M = 5
col_indices = range(N)

#defining variables
model_2.x = Var(col_indices, domain=NonNegativeIntegers)
model_2.a=Var(domain=NonNegativeIntegers)
model_2.b=Var(domain=NonNegativeIntegers)

#defining objective
model_2.objective = Objective(expr=summation(data_csvfile['Reach Quality'], model_2.x),sense=maximize)

#adding constraints
model_2.constraints = ConstraintList()

model_2.constraints.add(expr= model. _2.x[0] + model_2.x[1] + model_2.x[2]<= 75  )
model_2.constraints.add(expr= model_2.x[2] + model_2.x[3] +  model_2.x[4] <= 100  )
model_2.constraints.add(expr= model_2.x[0] + model_2.x[1] >= 10  )

model_2.constraints.add(summation(data_csvfile['Cost per advt'], model_2.x) <= 1500000)

model_2.constraints.add(summation(data_csvfile['Num Customers'], model_2.x) >= 900000)

#additional constraints
model_2.constraints.add(expr=model_2.x[0]+model_2.x[1]-model_2.x[2]-model_2.x[3]-model_2.x[4]-model_2.a+model_2.b==0)
model_2.constraints.add(model_2.a+model_2.b<=18)

model_2.pprint()



2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}

3 Var Declarations
    a : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    b : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeIntegers
    x : Size=5, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeIntegers
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers


In [17]:
opt_cbc_2 = SolverFactory('cbc')
result = opt_cbc_2.solve(model_2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)

Solver status: ok
Solver termination condition: optimal


2.4  By Approach 2
  
Solver status is OK  
Solver termination condition is optimal  
We are getting optimal solution by approach 2

In [18]:
print('\nObjective = ', model_2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('Num slots for medium ',data_csvfile['Medium'][i], model_2.x[i].value)

print("value of new variable a=",model_2.a.value)
print("value of new variable b=",model_2.b.value)

print('\nConstraints')
model_2.constraints.display()


Objective =  3378.0

Decision Variables
Num slots for medium  Websites 34.0
Num slots for medium  TV 0.0
Num slots for medium  DailyNewspaper 41.0
Num slots for medium  WeeklyMagazine 11.0
Num slots for medium  MonthlyMagazine 0.0
value of new variable a= 0.0
value of new variable b= 18.0

Constraints
constraints : Size=7
    Key : Lower    : Body       : Upper
      1 :     None :       75.0 :      75.0
      2 :     None :       52.0 :     100.0
      3 :     10.0 :       34.0 :      None
      4 :     None :  1485000.0 : 1500000.0
      5 : 900000.0 : 12095000.0 :      None
      6 :      0.0 :        0.0 :       0.0
      7 :     None :       18.0 :      18.0


2.5 By approach 1

Optimal objective value and value of decision variables and new introduced varible are shown above.

Constraint activity.  
Constraints 1,6 and 7 are active. Other constraints are inactive.

2.6 
The values of original decision variables x[i], obtained from both the approaches are exactly same.

2.7
New constraint is absolute value of difference in  "no. of newspapers" and "total no. of magazines of both types" is atleast 4.

i.e.   
$|x_{4} + x_{5}-x_{3} |$ >=4  

New formulation 

\begin{align}
\max \ 35 x_1 + 57 x_2 + 48 x_3 + 20 x_4 + 15 x_5, \\
{\rm{s.t.}} \; x_1 + x_2 + x_3 & \leq 75, \\
x_3 + x_4 + x_5 & \leq 100, \\
x_1 + x_2 & \geq 10, \\ 
30000 x_1 + 95000 x_2 + 10000 x_3 + 5000 x_4 + 4000 x_5 & \leq 1500000, \\
100000 x_1 + 500000 x_2 + 200000 x_3 + 45000 x_4 + 25000 x_5 & \geq 900000, \\
x_i & \geq 0, \quad i \in \{1,2,3,4,5\}. \\
\end{align}

New constraint
$|x_{4} + x_{5}-x_{3} |$ >=4  


This new formulation is not a linear program as it contains absolute value function which is not a linear function.  


This can be converted into linear programs by replacing the new constraint by following constraints.

$x_{4} + x_{5}-x_{3} $ >=4   
$x_{4} + x_{5}-x_{3} $ <=-4 